In [63]:
from ipyleaflet import Map, basemaps, basemap_to_tiles, Marker, Polygon, WidgetControl
from ipywidgets import IntSlider, ColorPicker, Button, link

import datetime

from functools import partial

t_str = datetime.datetime.now().strftime('%Y-%m-%d')
print(t_str)

m = Map(
    layers=(basemap_to_tiles(basemaps.NASAGIBS.ModisTerraTrueColorCR, t_str), ),
    center=(52.204793, 360.121558),
    zoom=2
)

domain_coords = []
polygon = None

def _update_domain_render():
    global polygon
    if polygon is not None:
        m.remove_layer(polygon)

    if len(domain_coords) > 1:  
        polygon = Polygon(
            locations=domain_coords,
            color="green",
            fill_color="green"
        )
        m.add_layer(polygon);
    else:
        polygon = None
        
def handle_move(marker, location, **kwargs):
    old_loc = marker.location
    new_loc = location
    idx = domain_coords.index(old_loc)
    domain_coords[idx] = new_loc
    _update_domain_render()
#    print(old_loc, location, marker.location)
        
marker_locs = {}

def handle_click(**kwargs):
    if kwargs.get('type') == 'click':
        loc = kwargs.get('coordinates')
        domain_coords.append(loc)
        marker = Marker(location=loc)
        marker.on_move(partial(handle_move, marker=marker))
        marker_locs[marker] = loc
        m.add_layer(marker)
        _update_domain_render()

m.on_interaction(handle_click)

def _clear_domain(*args, **kwargs):
    global domain_coords
    global marker_locs
    domain_coords = []
    for marker in marker_locs.keys():
        m.remove_layer(marker)
    marker_locs = {}
    _update_domain_render()
    
def _print_domain(*args, **kwargs):
    print(domain_coords)

button_reset = Button(description='reset')
button_reset.on_click(_clear_domain)
button_print = Button(description='print domain')
button_print.on_click(_print_domain)

m.add_control(WidgetControl(widget=button_print, position='bottomright'))
m.add_control(WidgetControl(widget=button_reset, position='bottomright'))

m

2019-07-03


Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …

In [59]:
from ipyleaflet import Map, basemaps, basemap_to_tiles, Marker, Polygon, WidgetControl
from ipywidgets import IntSlider, ColorPicker, Button, link

from functools import partial


class DomainPicker():
    def __init__(self):
        self.m = Map(
            layers=(basemap_to_tiles(basemaps.NASAGIBS.ModisTerraTrueColorCR, "2017-04-08"), ),
            center=(52.204793, 360.121558),
            zoom=2
        )

        self.domain_coords = []
        self.polygon = None
        self.marker_locs = {}
        
        self.m.on_interaction(self._handle_map_click)
        
        button_reset = Button(description='reset')
        button_reset.on_click(self._clear_domain)
        button_print = Button(description='print domain')
        button_print.on_click(self._print_domain)

        self.m.add_control(WidgetControl(widget=button_print, position='bottomright'))
        self.m.add_control(WidgetControl(widget=button_reset, position='bottomright'))
        
    @classmethod
    def create(cls):
        return cls().m

    def _update_domain_render(self):
        if self.polygon is not None:
            self.m.remove_layer(self.polygon)

        if len(self.domain_coords) > 1:  
            self.polygon = Polygon(
                locations=self.domain_coords,
                color="green",
                fill_color="green"
            )
            self.m.add_layer(self.polygon);
        else:
            self.polygon = None
        
    def _handle_marker_move(self, marker, location, **kwargs):
        old_loc = marker.location
        new_loc = location
        idx = self.domain_coords.index(old_loc)
        self.domain_coords[idx] = new_loc
        self._update_domain_render()

    def _handle_map_click(self, **kwargs):
        if kwargs.get('type') == 'click':
            loc = kwargs.get('coordinates')
            marker = Marker(location=loc)
            marker.on_move(partial(self._handle_marker_move, marker=marker))
            self.domain_coords.append(loc)
            self.marker_locs[marker] = loc
            self.m.add_layer(marker)
            self._update_domain_render()

    def _clear_domain(self, *args, **kwargs):
        self.domain_coords = []
        for marker in self.marker_locs.keys():
            self.m.remove_layer(marker)
        self.marker_locs = {}
        self._update_domain_render()

    def _print_domain(self, *args, **kwargs):
        print(self.domain_coords)


DomainPicker.create()

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …

In [68]:
import sys
sys.path.append('/home/earlcd/git-repos/convml_tt/convml_tt/data/sources/')

In [69]:
import domain_picker_ui

In [71]:
domain_picker_ui.DomainPicker(n_days_old_satimg=2).m

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …

[[-29.61167011519739, 288.43971990584396], [-55.02802211299251, 283.87004325288774], [-54.418929968658254, 249.42171156137132], [-28.998531814051795, 258.9125784559727]]
